In [1]:
# 📦 Importy
import os
import fitz  # pymupdf
import pandas as pd
import json
from tqdm.notebook import tqdm

# 🗂️ Ścieżki do folderów
RAW_DIR = "../data/raw"
PROCESSED_DIR = "../data/processed"

# 📌 Sprawdzenie, czy foldery istnieją
os.makedirs(PROCESSED_DIR, exist_ok=True)

# ✅ Podgląd plików w raw
pdf_files = [f for f in os.listdir(RAW_DIR) if f.lower().endswith(".pdf")]
print(f"Liczba plików PDF w raw: {len(pdf_files)}")
pdf_files[:5]  # pokaż pierwsze 5 plików


Liczba plików PDF w raw: 27


['Characterization of carbon nanotube filters and other carbonaceous materials by Raman spectroscopy—II study on dispersion and disorder parameters.pdf',
 'Jorio_2003_New_J._Phys._5_139.pdf',
 'PhysRevB.61.14095_b_wazna.pdf',
 'porównaine dwa lasery.pdf',
 'R1.pdf']

In [4]:
# 📦 Import tqdm (zwykły, nie notebook)
from tqdm import tqdm
import fitz  # pymupdf
import os
import pandas as pd
import json

# 📂 Ścieżki (te same co w pierwszej komórce)
RAW_DIR = "../data/raw"
PROCESSED_DIR = "../data/processed"

# ✅ Upewniamy się, że folder processed istnieje
os.makedirs(PROCESSED_DIR, exist_ok=True)

# Lista plików PDF
pdf_files = [f for f in os.listdir(RAW_DIR) if f.lower().endswith(".pdf")]
print(f"Liczba plików PDF w raw: {len(pdf_files)}")
pdf_files[:5]  # podgląd pierwszych 5 plików

# Lista do przechowywania metadanych
metadata_list = []

# Iteracja po PDF-ach
for pdf_file in tqdm(pdf_files, desc="Processing PDFs"):
    pdf_path = os.path.join(RAW_DIR, pdf_file)
    doc = fitz.open(pdf_path)
    
    # Wyciąganie tekstu ze stron
    pages_text = [doc[page_num].get_text() for page_num in range(len(doc))]
    
    # Metadata
    metadata = {
        "filename": pdf_file,
        "num_pages": len(doc),
        "text": pages_text
    }
    metadata_list.append(metadata)
    
    # Zapis JSON dla każdego PDF
    json_path = os.path.join(PROCESSED_DIR, pdf_file.replace(".pdf", ".json"))
    with open(json_path, "w", encoding="utf-8") as f:
        json.dump(metadata, f, ensure_ascii=False, indent=2)

# Zapis zbiorczy metadata CSV
df_metadata = pd.DataFrame([{"filename": m["filename"], "num_pages": m["num_pages"]} for m in metadata_list])
df_metadata.to_csv(os.path.join(PROCESSED_DIR, "metadata.csv"), index=False)

print("✅ PDF-y wczytane i zapisane jako JSON + metadata CSV")


Liczba plików PDF w raw: 27

















Processing PDFs: 100%|██████████| 27/27 [00:02<00:00,  9.87it/s]

✅ PDF-y wczytane i zapisane jako JSON + metadata CSV


In [5]:
import json

# Parametry chunkingu
CHUNK_SIZE = 500    # liczba znaków na chunk
OVERLAP = 100       # liczba znaków nakładki między chunkami

# Wczytanie wczytanych JSON-ów
pdf_json_files = [f for f in os.listdir(PROCESSED_DIR) if f.lower().endswith(".json") and f != "chunks.json"]

all_chunks = []

for pdf_file in tqdm(pdf_json_files, desc="Chunking PDFs"):
    json_path = os.path.join(PROCESSED_DIR, pdf_file)
    with open(json_path, "r", encoding="utf-8") as f:
        data = json.load(f)
    
    filename = data["filename"]
    text_pages = data["text"]
    
    # Łączymy wszystkie strony w jeden tekst
    full_text = "\n".join(text_pages)
    
    # Chunkowanie
    start = 0
    text_length = len(full_text)
    while start < text_length:
        end = min(start + CHUNK_SIZE, text_length)
        chunk_text = full_text[start:end]
        
        all_chunks.append({
            "filename": filename,
            "chunk_start": start,
            "chunk_end": end,
            "text": chunk_text
        })
        
        start += CHUNK_SIZE - OVERLAP

# Zapis wszystkich chunków do JSON
chunks_path = os.path.join(PROCESSED_DIR, "chunks.json")
with open(chunks_path, "w", encoding="utf-8") as f:
    json.dump(all_chunks, f, ensure_ascii=False, indent=2)

print(f"✅ Chunking zakończony! Liczba chunków: {len(all_chunks)}")



Chunking PDFs: 100%|██████████| 27/27 [00:00<00:00, 852.80it/s]

✅ Chunking zakończony! Liczba chunków: 3443


In [6]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import json
import os

# Parametry
MODEL_NAME = "all-MiniLM-L6-v2"  # szybki, lekki model
EMBEDDING_DIM = 384              # wymiar dla all-MiniLM-L6-v2

# 🔹 Wczytanie chunków
chunks_path = os.path.join(PROCESSED_DIR, "chunks.json")
with open(chunks_path, "r", encoding="utf-8") as f:
    chunks = json.load(f)

print(f"Liczba chunków do embeddingu: {len(chunks)}")

# 🔹 Utworzenie modelu embeddingów
model = SentenceTransformer(MODEL_NAME)

# 🔹 Tworzenie embeddings
texts = [c["text"] for c in chunks]
embeddings = model.encode(texts, show_progress_bar=True, batch_size=64)
embeddings = np.array(embeddings).astype("float32")

# 🔹 Tworzenie FAISS index
index = faiss.IndexFlatL2(EMBEDDING_DIM)  # L2 distance
index.add(embeddings)
print(f"✅ FAISS index utworzony. Liczba wektorów w indexie: {index.ntotal}")

# 🔹 Zapis FAISS index na dysku
faiss.write_index(index, os.path.join(PROCESSED_DIR, "faiss_index.index"))

# 🔹 Zapis chunks metadata (do retrieval)
chunks_meta_path = os.path.join(PROCESSED_DIR, "chunks_meta.json")
with open(chunks_meta_path, "w", encoding="utf-8") as f:
    json.dump(chunks, f, ensure_ascii=False, indent=2)

print("✅ Embeddings i FAISS index zapisane w data/processed/")


c:\Users\slast\miniconda3\envs\13_RAG\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Exception ignored in: <function tqdm.__del__ at 0x00000262EE69CE50>
Traceback (most recent call last):
  File "c:\Users\slast\miniconda3\envs\13_RAG\lib\site-packages\tqdm\std.py", line 1148, in __del__
    self.close()
  File "c:\Users\slast\miniconda3\envs\13_RAG\lib\site-packages\tqdm\notebook.py", line 277, in close
    self.disp(bar_style='danger', check_delay=False)
AttributeError: 'tqdm_notebook' object has no attribute 'disp'


Liczba chunków do embeddingu: 3443


c:\Users\slast\miniconda3\envs\13_RAG\lib\site-packages\huggingface_hub\file_download.py:130: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\slast\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Loading weights: 100%|██████████| 103/103 [00:00<00:00, 392.50it/s, Materializing param=pool

✅ FAISS index utworzony. Liczba wektorów w indexie: 3443
✅ Embeddings i FAISS index zapisane w data/processed/


In [7]:
import faiss
import numpy as np
import json
from sentence_transformers import SentenceTransformer

# 🔹 Parametry
TOP_K = 5  # ile najbardziej podobnych chunków
MODEL_NAME = "all-MiniLM-L6-v2"
EMBEDDING_DIM = 384

# 🔹 Wczytanie FAISS index
index_path = "../data/processed/faiss_index.index"
index = faiss.read_index(index_path)

# 🔹 Wczytanie metadata chunków
with open("../data/processed/chunks_meta.json", "r", encoding="utf-8") as f:
    chunks_meta = json.load(f)

# 🔹 Model embeddingów
model = SentenceTransformer(MODEL_NAME)

# 🔹 Funkcja do zapytania
def retrieve(query, top_k=TOP_K):
    query_embedding = model.encode([query]).astype("float32")
    distances, indices = index.search(query_embedding, top_k)
    
    results = []
    for i, idx in enumerate(indices[0]):
        chunk_info = chunks_meta[idx]
        results.append({
            "rank": i+1,
            "filename": chunk_info["filename"],
            "chunk_start": chunk_info["chunk_start"],
            "chunk_end": chunk_info["chunk_end"],
            "text": chunk_info["text"],
            "distance": float(distances[0][i])
        })
    return results

# 🔹 Przykładowe zapytanie
query = "Raman spectroscopy of carbon nanotubes"
results = retrieve(query)

for r in results:
    print(f"\n--- RANK {r['rank']} | File: {r['filename']} | Distance: {r['distance']:.4f} ---\n")
    print(r["text"][:500], "...")  # pokaż pierwsze 500 znaków chunka


Loading weights: 100%|██████████| 103/103 [00:00<00:00, 194.99it/s, Materializing param=pooler.dense.weight]                             
BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.



--- RANK 1 | File: R12.pdf | Distance: 0.2040 ---

of Raman scattering
processes relevant to carbon nanotubes are reviewed, and the theoretical foundations for these topics are presented.
The most common experimental techniques used to probe carbon nanotubes are summarized, followed by a review of
the novel experimental ﬁndings for each of the features in the ﬁrst order and second order Raman spectra for single
wall carbon nanotubes. These results are presented and discussed in connection with theoretical considerations.
Raman spectra for bundle ...

--- RANK 2 | File: Raman spectroscopy of carbon nanotubes.pdf | Distance: 0.2040 ---

of Raman scattering
processes relevant to carbon nanotubes are reviewed, and the theoretical foundations for these topics are presented.
The most common experimental techniques used to probe carbon nanotubes are summarized, followed by a review of
the novel experimental ﬁndings for each of the features in the ﬁrst order and second order Raman spectra for

In [9]:
# 🔹 Załaduj .env
from dotenv import load_dotenv
import os
load_dotenv()  # automatycznie szuka pliku .env w katalogu roboczym

# 🔹 Sprawdzenie klucza
if "OPENAI_API_KEY" not in os.environ:
    raise ValueError("Nie znaleziono OPENAI_API_KEY w zmiennych środowiskowych!")

# 🔹 Import OpenAI
from openai import OpenAI

# 🔹 Utworzenie klienta
client = OpenAI()  # bierze klucz z os.environ

# 🔹 Funkcja RAG
def rag_query(query, top_k=5):
    # 1️⃣ Retrieve z FAISS
    retrieved_chunks = retrieve(query, top_k=top_k)
    
    # 2️⃣ Połącz teksty w kontekst
    context = "\n\n".join([c["text"] for c in retrieved_chunks])
    
    # 3️⃣ Przygotowanie promptu
    prompt = f"Masz dostęp do następujących fragmentów publikacji naukowych:\n\n{context}\n\nOdpowiedz wyczerpująco na pytanie: {query}"
    
    # 4️⃣ Wywołanie LLM
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "Jesteś ekspertem w dziedzinie spektroskopii Ramana i nanostruktur węglowych."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=500
    )
    
    return response.choices[0].message.content

# 🔹 Przykład zapytania
query = "Jak Raman spectroscopy pozwala określić właściwości nanorurek węglowych?"
answer = rag_query(query)
print(answer)


Raman spectroscopy jest jedną z najskuteczniejszych technik do analizy nanorurek węglowych (SWNT - single-walled carbon nanotubes) ze względu na jej zdolność do dostarczania informacji o ich strukturze, właściwościach elektronicznych oraz dynamice. Oto kluczowe aspekty, które wyjaśniają, jak Raman spectroscopy umożliwia określenie właściwości nanorurek węglowych:

### 1. **Charakterystyka spektralna**
Raman spectroscopy pozwala na identyfikację charakterystycznych pasm (band), takich jak:
- **D (Disorder) band**: związane z defektami w strukturze węglowej, co pozwala na ocenę poziomu nieuporządkowania w nanorurkach.
- **G band**: odpowiada za akustyczne oraz optyczne plastyczne wzbudzenia (phonons) w sieci węglowej i jest użytecznym wskaźnikiem stanu materiału.
- **RBM (Radial Breathing Mode)**: pasmo to jest szczególnie wrażliwe na średnicę nanorurek węglowych. Przesunięcia częstości RBM są związane bezpośrednio z (n, m) chiralnością nanorurek, co umożliwia ich klasyfikację.

### 2. *